In [1]:
import numpy as np 
import numba as nb
import pandas as pd
import sklearn

In [2]:
def phi(c, p):
    r = np.linalg.norm(c-p)
    sigma = 1e-5
    return np.exp(-sigma * r**2)

In [3]:
def phi_prime(c, p):
    r = np.linalg.norm(c-p)
    sigma = 1e-5
    return -2*sigma*r*np.exp(-sigma * r**2)

$\nabla u$

In [4]:
def varphi(c, p):
    return phi_prime(c,p) * (p-c) / np.linalg.norm(p-c)

In [5]:
def Phi_matrix(C, B):
    t =  B.shape[0]
    k = C.shape[0]
    mat = np.zeros((3 * t, k))
    for i in range(t):
        varphi_i = varphi(C, B[i])
        mat[i] = varphi_i[:,0].copy()
        mat[i+ t] = varphi_i[:,1].copy()
        mat[i+2*t] = varphi_i[:,2].copy()
    return mat

In [6]:
def a(C, B, V, Phi):
    sol = np.linalg.lstsq(np.dot(Phi.T, Phi), np.dot(Phi.T,V.T.reshape(3 * B.shape[0])), rcond=None)
    # print(sol[1:])
    return sol[0]

In [7]:
def nabla_u(C, alpha, P):
    u = np.zeros_like(P)
    for i in range(len(P)):
        u[i] = np.dot(alpha, varphi(C, P[i]))
    return u

$\nabla \wedge w$

In [8]:
def A(C, B, Phi):
    t =  B.shape[0]
    k = C.shape[0]
    zero = np.zeros((t, k))
    phix = Phi[:t]
    phiy = Phi[t:2*t]
    phiz = Phi[2*t:]
    return np.block([[zero, -phiz, phiy], [phiz, zero, -phix], [-phiy, phiz, zero]])

In [9]:
def nabla_wedge_w(C, alpha, P):
    u = np.zeros_like(P)
    k = C.shape[0]
    a1 = alpha[:k]
    a2 = alpha[k:2*k]
    a3 = alpha[2*k:]
    for i in range(len(P)):
        partials = varphi(C,P[i])
        x_partial = partials[:,0]
        y_partial = partials[:,1]
        z_partial = partials[:,2]
        u[i]= [np.dot(a3, y_partial)-np.dot(a2,z_partial),
        np.dot(a1, z_partial)-np.dot(a3,x_partial),
        np.dot(a2, x_partial)-np.dot(a1,y_partial)]
    return u
    

h

In [10]:
def savedata(points, velocities, name="testing.csv"):
    data_dict ={
        "x": points[:,0],
        "y": points[:,1],
        "z": 25 * points[:,2],
        "Vx": velocities[:,0],
        "Vy": velocities[:,1],
        "Vz": velocities[:,2],
    }
    data = pd.DataFrame.from_dict(data_dict)
    data.to_csv(name)


In [ ]:
def get_decomp(file, save_dir="decomp"):
    res = file.split("like_")[-1].split(".txt")[0]
    df = pd.read_csv(file[8:], skiprows=6, sep="\s+")
    points = np.concatenate((df[["x(m)", "y(m)", "z_s(m)"]], df[["x(m)", "y(m)", "z_b(m)"]])) / 1000
    velocities = np.concatenate((df[["vx_s(m/a)", "vy_s(m/a)", "vz_s(m/a)"]], df[["vx_b(m/a)", "vy_b(m/a)", "vz_b(m/a)"]]))
    n_centers =2000
    centers = sklearn.cluster.BisectingKMeans(n_clusters=n_centers//2, random_state=0).fit(df[["x(m)", "y(m)", "z_s(m)"]]).cluster_centers_
    centers = np.concatenate((centers, sklearn.cluster.BisectingKMeans(n_clusters=n_centers//2, random_state=0).fit(df[["x(m)", "y(m)", "z_b(m)"]]).cluster_centers_)) / 1000
    Phi = Phi_matrix(centers, points)
    alpha = a(centers, points, velocities, Phi)
    u = nabla_u(centers,alpha, points)
    alpha = a(centers, points, velocities, A(centers,points,Phi))
    w = nabla_wedge_w(centers,alpha, points)
    savedata(points, velocities, name= f"{save_dir}\\{res} original.csv")
    savedata(points, u, name= f"{save_dir}\\{res} gradfree.csv")
    savedata(points, w, name= f"{save_dir}\\{res} divfree.csv")
    savedata(points, velocities - w - u, name= f"{save_dir}\\{res} harmonic.csv")
